In [1]:
import pygame
from environment_rl import make_env
from models import make_actor_critic

import torch
import torch.nn as nn
from tensordict.nn import TensorDictModule
from torchrl.objectives.value import GAE
from tensordict import TensorDict, TensorDictBase
from torchrl.modules import ProbabilisticActor, OneHotCategorical, ValueOperator
from torchrl.collectors import aSyncDataCollector, SyncDataCollector,  MultiSyncDataCollector

pygame.init()

pygame 2.5.2 (SDL 2.28.3, Python 3.9.19)
Hello from the pygame community. https://www.pygame.org/contribute.html


(5, 0)

In [2]:
env = make_env(transform=True)

In [3]:
actor, critic = make_actor_critic(env.action_spec)

In [4]:
# torch.save(actor, "./actor.pt")
# actor = torch.load("./actor.pt")

In [5]:
env.action_spec

OneHotDiscreteTensorSpec(
    shape=torch.Size([6]),
    space=DiscreteBox(n=6),
    device=cpu,
    dtype=torch.bool,
    domain=discrete)

In [6]:
obs = env.reset()

In [7]:
obs["obs"].shape

torch.Size([4, 140, 140])

In [8]:
actor(obs["obs"])

(tensor([[ 0.1114, -0.1134, -0.0116, -0.0187,  0.1237, -0.0972]],
        grad_fn=<AddmmBackward0>),
 tensor([[0, 0, 0, 0, 1, 0]]),
 tensor([-1.6713], grad_fn=<SqueezeBackward1>))

In [8]:
temp_data = env.rollout(10, actor)

In [9]:
gae = GAE(gamma=0.99, lmbda=0.95, value_network=critic,)

In [10]:
gae(temp_data)
# temp_data

TensorDict(
    fields={
        action: Tensor(shape=torch.Size([10, 1, 6]), device=cpu, dtype=torch.int64, is_shared=False),
        advantage: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.float32, is_shared=False),
        done: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.bool, is_shared=False),
        logits: Tensor(shape=torch.Size([10, 1, 6]), device=cpu, dtype=torch.float32, is_shared=False),
        next: TensorDict(
            fields={
                done: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                obs: Tensor(shape=torch.Size([10, 4, 140, 140]), device=cpu, dtype=torch.float32, is_shared=False),
                reward: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.float32, is_shared=False),
                state_value: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.float32, is_shared=False),
                step_count: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype

In [10]:
# actor(obs), 
critic(temp_data)

TensorDict(
    fields={
        action: Tensor(shape=torch.Size([10, 1, 6]), device=cpu, dtype=torch.int64, is_shared=False),
        done: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.bool, is_shared=False),
        logits: Tensor(shape=torch.Size([10, 1, 6]), device=cpu, dtype=torch.float32, is_shared=False),
        next: TensorDict(
            fields={
                done: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                obs: Tensor(shape=torch.Size([10, 4, 141, 240]), device=cpu, dtype=torch.float32, is_shared=False),
                reward: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.float32, is_shared=False),
                step_count: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.int64, is_shared=False),
                terminated: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.bool, is_shared=False)},
            batch_size=torch.Size([10]),
            device=None,
         

In [15]:
temp_data

TensorDict(
    fields={
        action: Tensor(shape=torch.Size([10, 1, 6]), device=cpu, dtype=torch.int64, is_shared=False),
        done: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.bool, is_shared=False),
        logits: Tensor(shape=torch.Size([10, 1, 6]), device=cpu, dtype=torch.float32, is_shared=False),
        next: TensorDict(
            fields={
                done: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                obs: Tensor(shape=torch.Size([10, 4, 141, 240]), device=cpu, dtype=torch.float32, is_shared=False),
                reward: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.float32, is_shared=False),
                step_count: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.int64, is_shared=False),
                terminated: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.bool, is_shared=False)},
            batch_size=torch.Size([10]),
            device=None,
         

In [12]:
# temp_data = env.rollout(1000, actor, )
# temp_data['next', 'reward']

In [11]:
temp_data['action'].shape

torch.Size([38, 1, 6])

In [14]:
temp_data["next", "obs"].shape

torch.Size([157, 4, 140, 140])

In [14]:
# collector = aSyncDataCollector(
#     env,
#     actor,
#     frames_per_batch=8,
#     total_frames=8,
#     split_trajs=False,
#     device="cpu"
# )

collector = MultiSyncDataCollector(
    create_env_fn=[make_env, make_env],
    policy=actor,
    total_frames=200,
    max_frames_per_traj=50,
    frames_per_batch=100,
    reset_at_each_iter=True
)

EOFError: 

: 

In [11]:
for i, data in enumerate(collector):
    pass

In [12]:
data

TensorDict(
    fields={
        action: Tensor(shape=torch.Size([8, 6]), device=cpu, dtype=torch.int64, is_shared=False),
        collector: TensorDict(
            fields={
                traj_ids: Tensor(shape=torch.Size([8]), device=cpu, dtype=torch.int64, is_shared=False)},
            batch_size=torch.Size([8]),
            device=cpu,
            is_shared=False),
        done: Tensor(shape=torch.Size([8, 1]), device=cpu, dtype=torch.bool, is_shared=False),
        logits: Tensor(shape=torch.Size([8, 6]), device=cpu, dtype=torch.float32, is_shared=False),
        next: TensorDict(
            fields={
                done: Tensor(shape=torch.Size([8, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                obs: Tensor(shape=torch.Size([8, 4, 141, 240]), device=cpu, dtype=torch.float32, is_shared=False),
                reward: Tensor(shape=torch.Size([8, 1]), device=cpu, dtype=torch.float32, is_shared=False),
                step_count: Tensor(shape=torch.Size([8, 1